# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Суховерхова Диана

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [ ]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-27 17:37:59--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-27 17:37:59--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0caa39599689728e1ce8888719.dl.dropboxusercontent.com/cd/0/inline/A2omifqxf7oAtbDCyI4Sduf0YfhxHWvRvDjSmPwSsvrSjXSW8PZ37X7_eiF5qJSsJns9nejX8LyAEXN7Ce4gtfyc7f_dTGPxQ7XCvZRHVEL6uMPqJcGsIvw2yVnWo6AKzKk/file# [following]
--2020-04-27 17:37:59--  https://uc0caa39599689728e1ce8888719.dl.dropboxusercontent.com/cd/0/inline/A2omifqxf7oAtbDCyI4Sduf0YfhxHWvRvDjSmPwSsvrSjXSW8PZ37X7_eiF5qJSsJns9nejX8LyAEXN7Ce4gtfyc7f_dTGPxQ7XCvZ

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [ ]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [ ]:
data.shape

(30000, 4)

In [ ]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train[:5]


array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [ ]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [ ]:
tok = WordPunctTokenizer()

def preprocess(text: str) -> str:
    return ' '.join(tok.tokenize(text.lower()))


for j in range(X_train.shape[0]):
  X_train[j][0] = preprocess(X_train[j][0]) 
  X_train[j][1] = preprocess(X_train[j][1]) 

for j in range(X_test.shape[0]):
  X_test[j][0] = preprocess(X_test[j][0]) 
  X_test[j][1] = preprocess(X_test[j][1]) 

In [ ]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [ ]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности

In [ ]:
def bow_voc(text: np.array) -> list:
    dct = {}
    k = 10000
    for j in range(text.shape[0]):
       for sent in text[j]:
         sent_spl = sent.split()
         for word in sent_spl:
          if word not in dct:
             dct[word] = 0
          else:
             dct[word] += 1
    list_d = list(dct.items())
    list_d.sort(key=lambda i: i[1], reverse=True)
    most_common = []
    dct_with_ind = {}
    for i in range(k):
      most_common.append(list_d[i][0])
      dct_with_ind[list_d[i][0]] = i
    return most_common


In [ ]:
bow_vocabulary = bow_voc(X_train)

In [ ]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [ ]:
def text_to_bow(text: str) -> np.array:

    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """   
    words = text.split()
    return np.asarray([words.count(word) for word in bow_vocabulary])
    return res 


In [ ]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [ ]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description

    ans = []
    for index, prod in enumerate(items):
      vect_bow = list(text_to_bow(prod[1]))
      ans.append(vect_bow)
    return np.array(ans)

In [ ]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [ ]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
X_train_bow_sparsed = csr_matrix(X_train_bow)
X_test_bow_sparsed = csr_matrix(X_test_bow)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_sparsed, y_train)
acc_sc = accuracy_score(bow_model.predict(X_test_bow_sparsed), y_test)
print(acc_sc)

assert accuracy_score(bow_model.predict(X_test_bow_sparsed), y_test) > 0.7

0.7011111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_sparsed, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_sparsed), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_sparsed), y_test) > 0.68

0.6835555555555556


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Результат: качество логистической регрессии - 70.1%, SVC - 68.4%

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [ ]:
import random

def items_to_bow_with_title(items: np.array) -> np.array:

    ans = []
    for index, prod in enumerate(items):
      vect_bow = list(text_to_bow(prod[0]) * random.random())
      ans.append(vect_bow)
    return np.array(ans)

In [ ]:
X_train_bow_with_title = np.hstack((X_train_bow, items_to_bow_with_title(X_train)))
X_test_bow_with_title = np.hstack((X_test_bow, items_to_bow_with_title(X_test)))

In [ ]:
X_train_bow_with_title_sparsed = csr_matrix(X_train_bow_with_title)
X_test_bow_with_title_sparsed = csr_matrix(X_test_bow_with_title)

In [ ]:
bow_model_title = LogisticRegression(max_iter=100).fit(X_train_bow_with_title_sparsed, y_train)
acc_sc_title = accuracy_score(bow_model_title.predict(X_test_bow_with_title_sparsed), y_test)
print(acc_sc_title)

0.7332222222222222


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model_title = LinearSVC(max_iter=70).fit(X_train_bow_with_title_sparsed, y_train)
print(accuracy_score(bow_model_title.predict(X_test_bow_with_title_sparsed), y_test))

0.7288888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Результат: качество обеих моделей улучшилось (на 4% для логистической регрессии и на 6% для SVC)

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [ ]:
from sklearn.preprocessing import normalize

X_train_bow_normalize = normalize(X_train_bow_with_title_sparsed)
X_test_bow_normalize = normalize(X_test_bow_with_title_sparsed)

In [ ]:
bow_model_title = LogisticRegression(max_iter=100).fit(X_train_bow_normalize, y_train)
acc_sc_title = accuracy_score(bow_model_title.predict(X_test_bow_normalize), y_test)
print(acc_sc_title)

0.6224444444444445


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model_title = LinearSVC(max_iter=70).fit(X_train_bow_normalize, y_train)
print(accuracy_score(bow_model_title.predict(X_test_bow_normalize), y_test))

0.763


Результат: качество логистической регрессии уменьшилось на 14%, а SVC - выросло на 4% (по сравнению с предыдущими моделями) \\
Это объясняется тем, что в логистической регресии нормализация проводилась отдельно для обучающей и тестирующей выборок.

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-27 11:11:21--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.242, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-27 11:11:22--  http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)... 5.45.220.16, 2a02:6b8:0:2002::17
Connecting to cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)|5.45.220.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [ ]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-7hy7m7pt
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-7hy7m7pt
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=7303eb6a5971903fc7f159351baf05d0dc61ac64b5b163543360be8c55596e2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n3sk7ra5/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [ ]:
from pymystem3 import Mystem

mystem = Mystem()

def preprocess_mystem(text: str) -> str:
    return ' '.join(mystem.lemmatize(text.lower()))


for j in range(X_train.shape[0]):
  X_train[j][0] = preprocess_mystem(X_train[j][0]) 
  X_train[j][1] = preprocess_mystem(X_train[j][1]) 

for j in range(X_test.shape[0]):
  X_test[j][0] = preprocess_mystem(X_test[j][0]) 
  X_test[j][1] = preprocess_mystem(X_test[j][1]) 


In [ ]:
bow_vocabulary_ms = bow_voc(X_train)

In [ ]:
def text_to_bow_ms(text: str) -> np.array:

    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """   
    words = text.split()
    return np.asarray([words.count(word) for word in bow_vocabulary_ms])
    return res 


def items_to_bow_ms(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    
    ans = []
    for index, prod in enumerate(items):
      vect_bow = list(text_to_bow_ms(prod[1]))
      ans.append(vect_bow)
    return np.array(ans)

    
def items_to_bow_with_title_ms(items: np.array) -> np.array:

    ans = []
    for index, prod in enumerate(items):
      vect_bow = list(text_to_bow_ms(prod[0]) * random.random())
      ans.append(vect_bow)
    return np.array(ans)

In [ ]:
X_train_bow_ms = items_to_bow_ms(X_train)
X_test_bow_ms = items_to_bow_ms(X_test)

In [ ]:
X_train_bow_with_title_ms = np.hstack((X_train_bow_ms, items_to_bow_with_title_ms(X_train)))
X_test_bow_with_title_ms = np.hstack((X_test_bow_ms, items_to_bow_with_title_ms(X_test)))

In [ ]:
X_train_bow_with_title_sparsed_ms = csr_matrix(X_train_bow_with_title_ms)
X_test_bow_with_title_sparsed_ms = csr_matrix(X_test_bow_with_title_ms)

In [ ]:
bow_model_title = LogisticRegression(max_iter=100).fit(X_train_bow_with_title_sparsed_ms, y_train)
acc_sc_title = accuracy_score(bow_model_title.predict(X_test_bow_with_title_sparsed_ms), y_test)
print(acc_sc_title)

0.7582222222222222


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model_title = LinearSVC(max_iter=70).fit(X_train_bow_with_title_sparsed_ms, y_train)
print(accuracy_score(bow_model_title.predict(X_test_bow_with_title_sparsed_ms), y_test))

0.7541111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Результат: качество обеих моделей улучшилось по сравнению с предыдущими (без нормализации данных): качество логистической регрессии выросло на 3%, SVC - также на 3%

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [ ]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
#   1. Сколько раз оно встретилось в title и description во всём X_train
#   2. В тексте скольких товаров встретилось это слово

from collections import Counter


def count_tfidf(matr: np.array):
  k = 10000
  text = []
  for i in range(matr.shape[0]):
    words = (matr[i][0] + ' ' + matr[i][1]).split()
    for word in words:
      text.append(word)
  most_comm = Counter(text)
  most_common = most_comm.most_common(k)

  texts = []
  for i in range(matr.shape[0]):
    hp = list(set((matr[i][0] + ' ' + matr[i][1]).split()))
    texts.append(hp)
  idf_most_com = [0] * k
  for j in range(k):
    word = most_common[j][0]
    idf_most_com[j] += sum([1 for i in texts if word in i])
  return (most_common, idf_most_com)

In [ ]:
count_arr = count_tfidf(X_train)

In [ ]:
from math import log

def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    k = 10000
    words = set(text.split())
    ans = [0] * k
    i = 0
    for i in range(k):
      tp = count_arr[0][i]
      if tp[0] in words:
        ans[i] += (tp[1] / len(count_arr[0])) * log(X_train.shape[0] / count_arr[1][i])
      i += 1
    return np.array(ans)

In [ ]:
def my_tfidf(matr: np.array):
  res = []
  for i in range(matr.shape[0]):
    res.append(text_to_tfidf(matr[i][0] + ' ' + matr[i][1]))
  return np.array(res)

In [ ]:
# Нормализуйте данные
from sklearn.preprocessing import normalize

X_train_normalize = normalize(my_tfidf(X_train))
X_test_normalize = normalize(my_tfidf(X_test))

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [ ]:
X_train_norm_sparsed = csr_matrix(X_train_normalize)
X_test_norm_sparsed = csr_matrix(X_test_normalize)

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_norm_sparsed, y_train)
acc_sc = accuracy_score(bow_model.predict(X_test_norm_sparsed), y_test)
print(acc_sc)

0.42744444444444446


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model = LinearSVC(max_iter=70).fit(X_train_norm_sparsed, y_train)
print(accuracy_score(bow_model.predict(X_test_norm_sparsed), y_test))

0.5575555555555556


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Результат: качество логистической регрессии составило 42.7%, а SVC - 55.6%

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [ ]:
 from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
def vect(matr: np.array):
  res = []
  for i in range(matr.shape[0]):
    res.append(matr[i][0] + ' ' + matr[i][1])
  return res

In [ ]:
vectorizer = HashingVectorizer(n_features=10000)
X_train_hash = vectorizer.transform(vect(X_train))
X_test_hash = vectorizer.transform(vect(X_test))

In [ ]:
X_train_hash_sparsed = csr_matrix(X_train_hash)
X_test_hash_sparsed = csr_matrix(X_test_hash)

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_hash_sparsed, y_train)
acc_sc = accuracy_score(bow_model.predict(X_test_hash_sparsed), y_test)
print(acc_sc)

0.7208888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model = LinearSVC(max_iter=70).fit(X_train_hash_sparsed, y_train)
print(accuracy_score(bow_model.predict(X_test_hash_sparsed), y_test))

0.7965555555555556


Результат: качество логистической регрессии - 72.1%, SVC - 79.7%

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [ ]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-27 17:55:30--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-27 17:55:30--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ee8c7bf7e8417329bc6b553e2.dl.dropboxusercontent.com/cd/0/inline/A2pUMw0lpN-nayREpYijUT6CyHPsYR6wiRf1QqMSZ4AveGOBG-BNDOkfmXO4RLiaOl2sm8VU_kkUQxmflu2bQbNATwYBgNtcVxkw6X-Rnbkrxg/file# [following]
--2020-04-27 17:55:30--  https://uc1ee8c7bf7e8417329bc6b553e2.dl.dropboxusercontent.com/cd/0/inline/A2pUMw0lpN-nayREpYijUT6CyHPsYR6wiRf1QqMSZ4AveGOBG-BNDOkfmXO4RLiaOl2sm8VU_kkUQxmflu2bQbNATwYBgNtcVxkw6X-Rnbkrxg/file
Resolving uc1ee8c7bf7e841732

In [ ]:
!tar xzfv /content/ru.tar.gz

ru.bin
ru.vec


In [ ]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [ ]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [ ]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [ ]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)

In [ ]:
def sent_and_emb(matr: np.array):
  res = []
  for i in range(matr.shape[0]):
    sent = matr[i][0] + ' ' + matr[i][1]
    res.append(sentence_embedding(sent))
  return np.array(res)

In [ ]:
X_train_emb = sent_and_emb(X_train)

In [ ]:
X_test_emb = sent_and_emb(X_test)

In [ ]:
X_train_emb_sparsed = csr_matrix(X_train_emb)
X_test_emb_sparsed = csr_matrix(X_test_emb)

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_emb_sparsed, y_train)
acc_sc = accuracy_score(bow_model.predict(X_test_emb_sparsed), y_test)
print(acc_sc)

0.5773333333333334


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
bow_model = LinearSVC(max_iter=70).fit(X_train_emb_sparsed, y_train)
print(accuracy_score(bow_model.predict(X_test_emb_sparsed), y_test))

0.5921111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Результат: качество логистической регрессии - 57.7%, SVC - 59.2%

### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (ссылка на них появится здесь)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.